# BEFORE

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from langchain.retrievers import EnsembleRetriever, BM25Retriever

In [2]:
load_dotenv()
# CHROMA_PATH = "../langchain_chroma_db"

True

In [3]:
chroma_db_path="./chroma_db1"
# collection_name="indonesian_docs"
model_name="LazarusNLP/all-indobert-base-v2"
chunk_size=500
chunk_overlap=100

In [4]:
embedding_model = HuggingFaceEmbeddings(
    model_name="LazarusNLP/all-indobert-base-v2",
    model_kwargs={"device": "cuda"},   # ubah ke "cuda" jika pakai GPU
    encode_kwargs={'normalize_embeddings': True}
)

d:\sistem\anaconda\envs\klien-yolo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load existing vectorstore if exists
if os.path.exists(chroma_db_path):
    try:
        vectorstore = Chroma(
            persist_directory=chroma_db_path,
            embedding_function=embedding_model,
            # collection_name=collection_name
        )
        print(f"Loaded existing vectorstore with {vectorstore._collection.count()} documents")
    except:
        print("Creating new vectorstore...")

print("Initialization complete!")

Loaded existing vectorstore with 34 documents
Initialization complete!


In [6]:
db = Chroma(
    persist_directory=chroma_db_path,
    embedding_function=embedding_model
)

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"]
    )

In [7]:
prompt_template = ChatPromptTemplate.from_template("""
Anda adalah AI asisten yang membahas mengenai tanaman anggur di Indonesia.
Gunakan data berikut untuk menjawab:

=== KONTEN DOKUMEN ===
{context}

=== PERTANYAAN PENGGUNA ===
{question}

Berikan jawaban:
- Akurat
- Bahasa Indonesia
- Mudah dipahami oleh semua kalangan
- Berbasis gejala dan hal lainnya yang relevan dengan konteks
- Jangan membuat jawaban yang tidak ada di konteks (jika tidak ada, katakan 'Maaf, saya tidak menemukan informasi yang relevan, silakan bertanya pada sumber lain.')
""")

In [17]:
vector_retriever = db.as_retriever(search_kwargs={"k": 2})

In [18]:
# all_docs = db.get(include=["documents"])["documents"]
# Ambil semua data dari Chroma
raw = db.get(include=["documents"])

all_docs = raw["documents"]  # dokumen berupa list of strings


bm25_retriever = BM25Retriever.from_texts(all_docs)
bm25_retriever.k = 3  # jumlah dokumen yang diambil

In [19]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]  # Equal weight to vector and keyword search
)

In [20]:
def ask_hybrid(question: str):
    # hybrid search
    docs = hybrid_retriever.invoke(question)
    context = "\n\n".join([d.page_content for d in docs])

    chain = prompt_template | model
    response = chain.invoke({
        "context": context,
        "question": question
    })
    print(response.content)

In [57]:
def ask_vector(question: str, k: int = 5, min_score: float = 0.2):
    """
    RAG dengan vector search saja (tanpa BM25)
    Menggunakan similarity_search_with_relevance_scores.
    
    Params:
      - question: pertanyaan user
      - k: jumlah dokumen teratas yang diambil
      - min_score: minimum relevansi (semakin kecil semakin longgar)
    """
    
    # Vector search + relevance score
    results = db.similarity_search_with_relevance_scores(question, k=k)

    # Filter berdasarkan threshold score
    filtered_docs = [
        doc for doc, score in results if score >= min_score
    ]

    # Jika tidak ada dokumen relevan
    if len(filtered_docs) == 0:
        context = "Tidak ada konteks yang relevan ditemukan."
    else:
        context = "\n\n".join([d.page_content for d in filtered_docs])

    # Forward ke prompt dan model
    chain = prompt_template | model
    response = chain.invoke({
        "context": context,
        "question": question
    })

    return response.content


In [20]:
jawaban = ask("Apa penyebab daun anggur terdapat bercak coklat?")
print(jawaban.content)

Bapak/Ibu Petani,

Jika daun anggur Bapak/Ibu terdapat bercak coklat, ada dua kemungkinan penyebab utamanya berdasarkan kondisi di Indonesia:

1.  **Penyakit Brown Spot**
    *   **Penyebab:** Penyakit ini disebabkan oleh jamur bernama *Cercospora spp.*
    *   **Gejala:** Bercak coklat yang muncul biasanya kecil, bentuknya bulat, dan uniknya bagian tengah bercak lebih terang warnanya dibanding bagian pinggirnya yang lebih gelap. Jika parah, daun bisa rontok lebih cepat.
    *   **Kondisi yang disukai:** Penyakit ini sering muncul di kebun anggur dataran rendah yang curah hujannya tinggi (misalnya di Jawa Barat dan Lampung), terutama saat suhu hangat (sekitar 28–32°C) dan daun anggur sering basah.
    *   **Solusi Praktis:**
        *   **Sanitasi:** Bersihkan daun-daun anggur yang sudah gugur di bawah pohon.
        *   **Pengaturan Jarak:** Atur jarak tanam anggur agar tidak terlalu rapat, supaya sirkulasi udara lebih baik.
        *   **Fungisida:** Jika sudah parah, bisa menggunaka

In [22]:
jawab = ask_hybrid("Apa penyebab daun anggur terdapat bercak coklat?")

Bercak coklat pada daun anggur di Indonesia bisa disebabkan oleh dua jenis penyakit utama, yaitu:

1.  **Brown spot (Bercak Coklat)**
    *   **Penyebab:** Penyakit ini disebabkan oleh jamur *Cercospora spp.*
    *   **Gejala:** Muncul sebagai bercak coklat kecil berbentuk bulat pada daun. Ciri khasnya adalah bagian tengah bercak lebih terang, sedangkan tepinya lebih gelap.
    *   **Kondisi yang mendukung:** Penyakit ini umum di kebun anggur dataran rendah dengan curah hujan tinggi, seperti di Jawa Barat dan Lampung. Berkembang cepat pada suhu hangat (28–32°C) dan daun yang sering basah.
    *   **Dampak:** Jika infeksi parah, daun bisa rontok lebih cepat, mengurangi kemampuan fotosintesis tanaman, dan menurunkan hasil panen.

2.  **Leaf Blight (Hawar Daun)**
    *   **Penyebab:** Penyakit ini disebabkan oleh jamur *Pseudocercospora spp.*
    *   **Gejala:** Awalnya muncul sebagai bercak yang bentuknya tidak beraturan, berwarna coklat gelap hingga ungu. Seiring waktu, bercak ini akan 

In [21]:
jawab = ask_hybrid("Apa penyakit yang disebabkan oleh jamur Cercospora spp. pada tanaman anggur?")

Penyakit yang disebabkan oleh jamur *Cercospora spp.* pada tanaman anggur adalah **Brown spot**.

Penyakit ini sering ditemukan di kebun anggur dataran rendah dengan curah hujan tinggi, seperti di Jawa Barat dan Lampung.

**Gejala yang muncul:**
*   Pada daun, terlihat bercak coklat kecil berbentuk bulat dengan bagian tengah yang lebih terang dan tepi yang lebih gelap.
*   Jika infeksi parah, daun bisa rontok lebih cepat, sehingga mengurangi proses fotosintesis dan akhirnya menurunkan hasil panen anggur.

Penyakit Brown spot ini berkembang pesat pada suhu hangat (sekitar 28–32°C) dan jika daun anggur sering basah.

**Untuk pengendaliannya, bisa dilakukan dengan cara:**
*   Membersihkan daun-daun yang gugur (sanitasi).
*   Mengatur jarak tanam agar tidak terlalu rapat.
*   Menggunakan fungisida berbahan aktif klorotalonil atau difenokonazol.


In [21]:
jawab = ask("Apa penyakit yang disebabkan oleh jamur Cercospora spp. pada tanaman anggur?")
print(jawab.content)

Bapak/Ibu petani,

Penyakit anggur yang disebabkan oleh jamur *Cercospora spp.* adalah **Brown Spot** atau bercak coklat pada daun.

**Gejala yang perlu diperhatikan:**
*   Muncul bercak-bercak kecil, bulat, dan berwarna coklat pada daun.
*   Bagian tengah bercak biasanya lebih terang, sedangkan tepinya lebih gelap.
*   Jika serangan parah, daun bisa rontok lebih cepat, mengurangi kemampuan tanaman berfotosintesis, dan akhirnya menurunkan hasil panen anggur.

Penyakit ini sering muncul di daerah dataran rendah dengan curah hujan tinggi, seperti Jawa Barat dan Lampung, terutama saat suhu hangat (sekitar 28–32°C) dan daun sering basah.

**Solusi praktis untuk pengendalian:**
1.  **Sanitasi:** Bersihkan dan buang daun-daun yang sudah gugur di sekitar tanaman.
2.  **Pengaturan Jarak Tanam:** Pastikan ada jarak yang cukup antar tanaman agar sirkulasi udara baik dan daun tidak terlalu lembap.
3.  **Penggunaan Fungisida:** Jika diperlukan, gunakan fungisida dengan bahan aktif seperti **klorot

In [22]:
jawab = ask("Apa penyakit yang disebabkan oleh jamur Cercospora spp. pada tanaman anggur?")
print(jawab.content)

Bapak/Ibu petani,

Penyakit yang disebabkan oleh jamur *Cercospora spp.* pada tanaman anggur adalah **Brown spot** (atau bercak coklat).

**Gejala yang bisa Bapak/Ibu lihat:**
Di daun anggur akan muncul bercak-bercak kecil berbentuk bulat berwarna coklat. Ciri khasnya, bagian tengah bercak ini biasanya lebih terang warnanya, sementara bagian pinggirnya lebih gelap. Kalau sudah parah, daun anggur bisa rontok lebih cepat dari biasanya, sehingga tanaman tidak bisa berfotosintesis (membuat makanan) dengan baik dan ini bisa menurunkan hasil panen anggur Bapak/Ibu. Penyakit ini sering ditemukan di kebun anggur dataran rendah dengan curah hujan tinggi dan cepat berkembang di suhu hangat (sekitar 28–32°C) serta daun yang sering basah.

**Solusi Praktis untuk Pengendalian:**
1.  **Sanitasi:** Bersihkan secara rutin daun-daun anggur yang sudah gugur di bawah pohon dan buang agar sumber penyakit tidak menyebar.
2.  **Jarak Tanam:** Pastikan jarak tanam anggur tidak terlalu rapat. Ini penting agar

In [58]:
ask_vector("Apa itu anggur")

'Maaf, saya tidak menemukan informasi yang relevan mengenai definisi umum "anggur" dalam konteks yang diberikan. Informasi yang tersedia lebih fokus pada varietas anggur yang dibudidayakan di Indonesia dan cara pemupukannya. Silakan bertanya pada sumber lain jika Anda membutuhkan definisi umum tentang anggur.'

In [28]:
ask("Siapa soekarno?")

Maaf, saya tidak menemukan informasi yang relevan mengenai "Soekarno" dalam data yang saya miliki tentang budidaya anggur.


In [35]:
ask("Siapa joko widodo?")

Maaf, saya tidak menemukan informasi mengenai "Joko Widodo" dalam konteks data yang diberikan.


In [36]:
ask("fufufafa")

Maaf, saya tidak menemukan informasi yang relevan dengan pertanyaan "fufufafa". Mohon ajukan pertanyaan mengenai tanaman anggur yang lebih spesifik.


In [43]:
ask("fufufafa")

Maaf, saya tidak menemukan informasi yang relevan dengan pertanyaan Anda. Silakan bertanya dengan pertanyaan yang lebih spesifik mengenai tanaman anggur di Indonesia.


In [53]:
ask_hybrid("Apa itu penyakit dengan nama latin Erysiphe necator?")

Penyakit yang disebabkan oleh jamur *Erysiphe necator* adalah **Powdery mildew** atau embun tepung.

**Gejala:**
Anda dapat mengenali penyakit ini dari tanda-tanda berikut:
*   Munculnya lapisan seperti tepung berwarna putih di permukaan daun, tunas, dan buah anggur.
*   Daun bisa terlihat melintir atau berubah bentuk.
*   Pertumbuhan tanaman menjadi terhambat.
*   Buah anggur bisa pecah sebelum matang.
*   Pada daun muda, sering terlihat gejala klorosis (menguning) yang disebut remaja klorosis.

**Solusi Praktis (Pengendalian):**
Untuk mengatasi Powdery mildew, Anda bisa melakukan beberapa hal berikut:
*   **Pangkas** tunas atau bagian tanaman yang sudah terinfeksi untuk mencegah penyebaran.
*   **Tingkatkan sirkulasi udara** di sekitar tanaman anggur. Ini bisa dilakukan dengan penataan jarak tanam yang baik atau pemangkasan daun yang terlalu rimbun.
*   Gunakan **sulfur (belerang)** sebagai salah satu cara pengendalian.
*   Jika diperlukan, gunakan fungisida dengan bahan aktif sepert

In [54]:
ask_vector("Apa itu penyakit dengan nama latin Erysiphe necator?")

'Maaf, saya tidak menemukan informasi mengenai penyakit dengan nama latin *Erysiphe necator* dalam konteks yang diberikan. Informasi yang tersedia hanya membahas mengenai pemupukan tanaman anggur berdasarkan fase pertumbuhannya. Silakan bertanya pada sumber lain untuk informasi mengenai penyakit tersebut.'

In [ ]:
ask_hybrid("Apa itu penyakit dengan nama latin Erysiphe necator?")

Penyakit yang disebabkan oleh jamur *Erysiphe necator* adalah **Powdery Mildew** atau sering disebut penyakit **Embun Tepung**.

Berikut adalah penjelasan mengenai penyakit ini:

**Gejala:**
*   Anda akan melihat lapisan seperti tepung berwarna putih di permukaan daun, tunas, dan buah anggur.
*   Daun yang terinfeksi bisa mengerut atau berubah bentuk.
*   Pertumbuhan tanaman anggur akan terhambat.
*   Buah anggur bisa pecah sebelum matang.
*   Pada daun muda, sering muncul bercak kekuningan (klorosis).

**Faktor Pemicu:**
Penyakit ini mudah berkembang pada kondisi cuaca kering-lembab dengan suhu antara 20–30°C. Daerah seperti Jawa Timur, NTB, dan NTT sering mengalami masalah ini.

**Solusi Praktis (Pengendalian):**
*   **Pemangkasan:** Pangkas tunas atau bagian tanaman yang sudah terinfeksi untuk mengurangi penyebaran.
*   **Sirkulasi Udara:** Pastikan kebun anggur Anda memiliki sirkulasi udara yang baik. Ini bisa dilakukan dengan penjarangan daun atau pengaturan jarak tanam.
*   **Pen

In [59]:
ask_hybrid("Apa itu penyakit dengan nama latin Erysiphe necatpr?")

Penyakit yang disebabkan oleh jamur dengan nama latin *Erysiphe necator* adalah **Powdery mildew** (embun tepung).

Berikut adalah informasi mengenai penyakit ini:

*   **Penyebab**: Penyakit ini disebabkan oleh jamur *Erysiphe necator*.
*   **Kondisi Favorit**: Powdery mildew berkembang baik pada kondisi kering–lembap dengan suhu sekitar 20–30°C. Penyakit ini umum ditemukan di daerah seperti Jawa Timur, NTB, dan NTT.
*   **Gejala**:
    *   Munculnya lapisan seperti tepung berwarna putih pada permukaan daun, tunas, dan buah.
    *   Daun bisa mengalami perubahan bentuk (distorsi) atau keriting.
    *   Pertumbuhan tanaman anggur akan terhambat.
    *   Buah dapat pecah sebelum matang.
    *   Seringkali muncul bercak kekuningan (klorosis) pada daun yang masih muda.
*   **Solusi Praktis (Pengendalian)**:
    *   **Pemangkasan**: Pangkas tunas atau bagian tanaman yang sudah terinfeksi untuk mengurangi penyebaran jamur.
    *   **Sirkulasi Udara**: Tingkatkan sirkulasi udara di sekitar t

In [60]:
ask_hybrid("Mengapa muncul lapisan seperti tepung pada daun tanaman saya?")

Berdasarkan gejala yang Anda sebutkan, yaitu munculnya lapisan seperti tepung pada daun tanaman anggur, kemungkinan besar tanaman Anda terserang penyakit **Powdery Mildew** atau embun tepung.

**Apa itu Powdery Mildew?**
Penyakit ini disebabkan oleh jamur bernama *Erysiphe necator*. Jamur ini membentuk lapisan putih seperti tepung pada permukaan daun, tunas, bahkan buah anggur Anda.

**Mengapa bisa muncul?**
Powdery mildew sering berkembang dengan baik pada kondisi cuaca yang kering-lembap dengan suhu sekitar 20-30°C. Penyakit ini cukup umum terjadi di daerah seperti Jawa Timur, NTB, dan NTT.

**Selain lapisan tepung, gejala lain yang mungkin terlihat:**
*   Daun bisa terlihat melintir atau tidak normal bentuknya.
*   Pertumbuhan tanaman bisa terhambat.
*   Bahkan buah bisa pecah sebelum matang.
*   Kadang, daun muda juga bisa terlihat menguning (remaja klorosis).

**Apa yang bisa Anda lakukan? (Solusi Praktis)**
1.  **Pangkas bagian yang terinfeksi:** Segera potong tunas atau daun yan

In [61]:
ask_hybrid("di kondisi shuh seperti apa penyakit powdery mildew muncul?")

Penyakit powdery mildew pada tanaman anggur di Indonesia berkembang baik pada suhu antara **20–30°C**, biasanya dalam kondisi lingkungan yang kering–lembap.

Gejala utama yang bisa Anda perhatikan adalah munculnya lapisan tepung putih pada permukaan daun, tunas, dan buah. Daun bisa terlihat cacat (distorsi) dan buah pecah sebelum matang. Untuk pengendalian praktisnya, Anda bisa memangkas tunas yang sudah terinfeksi dan memastikan sirkulasi udara di sekitar tanaman anggur Anda baik.


In [ ]:
ask_hybrid("apa saja varietas tanaman anggur?")

Berdasarkan informasi yang tersedia, berikut adalah beberapa varietas tanaman anggur yang umum dibudidayakan di Indonesia:

1.  **Anggur Red Globe:** Varietas ini memiliki buah yang besar, berwarna merah, dan rasanya manis. Sangat cocok dibudidayakan di dataran rendah.
2.  **Anggur Black Sapphire:** Anggur ini dikenal dengan bentuk buahnya yang lonjong, berwarna hitam pekat, dan rasanya sangat manis.
3.  **Anggur Jupiter:** Keunggulan varietas Jupiter adalah buahnya yang tanpa biji (seedless) dan memiliki produktivitas yang tinggi.
4.  **Anggur Ninel:** Ini adalah varietas yang cukup populer, terutama di daerah Jawa Timur.
5.  **Anggur Akademik Avidzba:** Varietas ini sangat unggul dan cocok untuk tumbuh di iklim yang lembap.
6.  **Anggur Baikonur:** Dikenal karena ketahanannya terhadap panas dan sangat sesuai untuk ditanam di pekarangan rumah.
7.  **Anggur Transfiguration:** Buah dari varietas ini memiliki biji yang sedikit, rasanya manis, dan ukurannya besar.
8.  **Anggur Kadota:** I

In [65]:
ask_hybrid("apa saja manfaat tanaman anggur?")

Berdasarkan informasi yang diberikan, berikut adalah manfaat dari tanaman anggur:

Buah anggur dapat dimanfaatkan dengan berbagai cara, antara lain:
*   Dimakan langsung dalam keadaan segar.
*   Diolah menjadi kismis.
*   Dijadikan jus.
*   Diolah menjadi jelly.
*   Dijadikan wine.


# AFTER

In [23]:
chroma_db_path1="./chroma_db"
# collection_name="indonesian_docs"
model_name="LazarusNLP/all-indobert-base-v2"
chunk_size=500
chunk_overlap=100

In [24]:
# Load existing vectorstore if exists
if os.path.exists(chroma_db_path1):
    try:
        vectorstore = Chroma(
            persist_directory=chroma_db_path1,
            embedding_function=embedding_model,
            # collection_name=collection_name
        )
        print(f"Loaded existing vectorstore with {vectorstore._collection.count()} documents")
    except:
        print("Creating new vectorstore...")

print("Initialization complete!")

Loaded existing vectorstore with 17 documents
Initialization complete!


In [25]:
vectorstore = Chroma(
    persist_directory=chroma_db_path1,
    embedding_function=embedding_model
)

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"]
    )